In [5]:
# !pip install scikit-learn
# !pip install pickle



In [6]:
from sklearn import linear_model
import pickle

In [7]:
# model = pickle.load(open('logreg_model_3months.sav', 'rb'))

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn import linear_model, metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
from datetime import datetime, timedelta
from pylab import rcParams
import pickle
import importlib
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os

from prepare_data import DataLoader
from evaluate_results import EvaluateResults

In [9]:
os.chdir('/Users/Hp/VS CODE/Stock market crash prediction')

In [16]:
import numpy as np
import pandas as pd
from datetime import timedelta

class DataLoader():
    def __init__(self, datasets_original, dataset_names):
        self.num_datasets = len(datasets_original)
        self.datasets_original = datasets_original
        self.dataset_names = dataset_names
        self.datases_revised = None
        self.drawdowns = None
        self.crashes = None
    
    def get_data_revised(self, crash_thresholds):
        datasets = []
        print(datasets_original)
        
        for d in self.datasets_original:

            data_original = pd.read_csv(d,index_col='Date')
            print('csv_ read')
            print(data_original)
#             print(data_original.head(5))

        
            data_original.index = pd.to_datetime(data_original.index, format='%Y-%m-%d')
            list(data_original.columns)
            print(data_original)
            data_norm = data_original['Close'] / data_original['Close'][-1]
            data_ch = data_original['Close'].pct_change()
            window = 10
            data_vol = data_original['Close'].pct_change().rolling(window).std()
            data = pd.concat([data_original['Close'], data_norm, data_ch, data_vol], axis=1).dropna()
            data.columns = ['price', 'norm', 'ch', 'vol']
            datasets.append(data)
        self.drawdowns = []
        self.crashes = []
        for df, ct in zip(datasets, crash_thresholds):
            pmin_pmax = (df['price'].diff(-1) > 0).astype(int).diff()
            pmax = pmin_pmax[pmin_pmax == 1]
            pmin = pmin_pmax[pmin_pmax == -1]
            # make sure drawdowns start with pmax, end with pmin:
            if pmin.index[0] < pmax.index[0]:
                pmin = pmin.drop(pmin.index[0])
            if pmin.index[-1] < pmax.index[-1]:
                pmax = pmax.drop(pmax.index[-1])
            D = (np.array(df['price'][pmin.index]) - np.array(df['price'][pmax.index]))/ np.array(df['price'][pmax.index])
            d = {'Date':pmax.index, 'drawdown':D, 'd_start': pmax.index, 'd_end': pmin.index}
            df_d = pd.DataFrame(d).set_index('Date')
            df_d.index = pd.to_datetime(df_d.index, format='%Y-%m-%d')
            df_d = df_d.reindex(df.index).fillna(0)
            df_d = df_d.sort_values(by='drawdown')
            df_d['rank'] = list(range(1,df_d.shape[0]+1))
            self.drawdowns.append(df_d)
            df_d = df_d.sort_values(by='Date')
            df_c = df_d[df_d['drawdown'] < ct]
            df_c.columns = ['drawdown', 'crash_st', 'crash_end', 'rank']
            self.crashes.append(df_c)
        self.datasets_revised = []  
        for i in range(len(datasets)):
            self.datasets_revised.append(pd.concat([datasets[i], self.drawdowns[i]], axis=1))
        return self.datasets_revised, self.crashes
    
    def get_dfs_xy(self, months):
    ### dfs_xy: dataframe for each dataset x (columns 0:-1) and  y (column -1)     
        dfs_x, dfs_y = [], []
        for df, c in zip(self.datasets_revised, self.crashes):
            df['ch'] = df['ch'] / abs(df['ch']).mean()
            df['vol'] = df['vol'] / abs(df['vol']).mean()
            xy = {}
            for date in df.index[252:-126]: # <--subtract 126 days in the end
                xy[date] = list([df['ch'][(date-timedelta(5)):date].mean()])
                xy[date].append(df['ch'][(date-timedelta(10)):(date-timedelta(5))].mean())
                xy[date].append(df['ch'][(date-timedelta(15)):(date-timedelta(10))].mean())
                xy[date].append(df['ch'][(date-timedelta(21)):(date-timedelta(15))].mean())
                xy[date].append(df['ch'][(date-timedelta(42)):(date-timedelta(21))].mean())
                xy[date].append(df['ch'][(date-timedelta(63)):(date-timedelta(42))].mean())
                xy[date].append(df['ch'][(date-timedelta(126)):(date-timedelta(63))].mean())
                xy[date].append(df['ch'][(date-timedelta(252)):(date-timedelta(126))].mean())
                xy[date].append(df['vol'][(date-timedelta(5)):date].mean())
                xy[date].append(df['vol'][(date-timedelta(10)):(date-timedelta(5))].mean())
                xy[date].append(df['vol'][(date-timedelta(15)):(date-timedelta(10))].mean())
                xy[date].append(df['vol'][(date-timedelta(21)):(date-timedelta(15))].mean())
                xy[date].append(df['vol'][(date-timedelta(42)):(date-timedelta(21))].mean())
                xy[date].append(df['vol'][(date-timedelta(63)):(date-timedelta(42))].mean())
                xy[date].append(df['vol'][(date-timedelta(126)):(date-timedelta(63))].mean())
                xy[date].append(df['vol'][(date-timedelta(252)):(date-timedelta(126))].mean())
                xy[date] = xy[date] + [max([date <= c and date + timedelta(month * 21) > c \
                          for c in c['crash_st']]) for month in months]
            df_xy = pd.DataFrame.from_dict(xy, orient='index').dropna()
            df_x = df_xy.iloc[:, :-len(months)]
            df_y = df_xy.iloc[:, -len(months):]
            dfs_x.append(df_x)
            dfs_y.append(df_y)
        return dfs_x, dfs_y
    
    def get_dfs_xy_predict(self, months):
    ### dfs_xy: dataframe for each dataset x (columns 0:-1) and  y (column -1)     
        dfs_x, dfs_y = [], []
        for df, c in zip(self.datasets_revised, self.crashes):
            df['ch'] = df['ch'] / abs(df['ch']).mean()
            df['vol'] = df['vol'] / abs(df['vol']).mean()
            xy = {}
            for date in df.index: # <--subtract 126 days in the end
                xy[date] = list([df['ch'][(date-timedelta(5)):date].mean()])
                xy[date].append(df['ch'][(date-timedelta(10)):(date-timedelta(5))].mean())
                xy[date].append(df['ch'][(date-timedelta(15)):(date-timedelta(10))].mean())
                xy[date].append(df['ch'][(date-timedelta(21)):(date-timedelta(15))].mean())
                xy[date].append(df['ch'][(date-timedelta(42)):(date-timedelta(21))].mean())
                xy[date].append(df['ch'][(date-timedelta(63)):(date-timedelta(42))].mean())
                xy[date].append(df['ch'][(date-timedelta(126)):(date-timedelta(63))].mean())
                xy[date].append(df['ch'][(date-timedelta(252)):(date-timedelta(126))].mean())
                xy[date].append(df['vol'][(date-timedelta(5)):date].mean())
                xy[date].append(df['vol'][(date-timedelta(10)):(date-timedelta(5))].mean())
                xy[date].append(df['vol'][(date-timedelta(15)):(date-timedelta(10))].mean())
                xy[date].append(df['vol'][(date-timedelta(21)):(date-timedelta(15))].mean())
                xy[date].append(df['vol'][(date-timedelta(42)):(date-timedelta(21))].mean())
                xy[date].append(df['vol'][(date-timedelta(63)):(date-timedelta(42))].mean())
                xy[date].append(df['vol'][(date-timedelta(126)):(date-timedelta(63))].mean())
                xy[date].append(df['vol'][(date-timedelta(252)):(date-timedelta(126))].mean())
                xy[date] = xy[date] + [max([date <= c and date + timedelta(month * 21) > c \
                          for c in c['crash_st']]) for month in months]
            df_xy = pd.DataFrame.from_dict(xy, orient='index').dropna()
            df_x = df_xy.iloc[:, :-len(months)]
            df_y = df_xy.iloc[:, -len(months):]
            dfs_x.append(df_x)
            dfs_y.append(df_y)
        return dfs_x, dfs_y

    def get_train_test(self, dfs_x, dfs_y, datasets, test_data):
        for i, name in enumerate(datasets):
            if name == test_data:
                index = i
        dfs_x_copy = list(dfs_x)
        dfs_y_copy = list(dfs_y)
        np_x_test = None
        np_y_test = None
        if test_data:
            df_x_test = dfs_x_copy.pop(index)
            df_y_test = dfs_y_copy.pop(index)
            np_x_test = np.array(df_x_test)
            np_y_test = np.array(df_y_test)
        np_x_train = np.concatenate(([np.array(x) for x in dfs_x_copy]))
        np_y_train = np.concatenate(([np.array(y) for y in dfs_y_copy]))
        return np_x_train, np_y_train, np_x_test, np_y_test

    def split_results(self, df_combined, dfs_xy, dataset_names, test_data, y_pred_t_bin, \
                      y_pred_tr_bin, y_train, y_test):
        df_combined = [dfc.reindex(dfs.index) for dfc, dfs in zip(df_combined, dfs_xy)]
        dfs_predict = []
        n = 0
        for df, name in zip(df_combined, dataset_names):
            if name == test_data:
                df['y'] = y_test
                df['y_pred'] = y_pred_t_bin
                dfs_predict.append(df)
            else:
                df['y'] = y_train[n:n+df.shape[0]]
                df['y_pred'] = y_pred_tr_bin[n:n+df.shape[0]]
                dfs_predict.append(df)
                n += df.shape[0]
        return dfs_predict

In [ ]:

datasets_original = ['^GSPC.csv', '^N225.csv', 'SSE.csv','^HSI.csv', '^BSESN.csv',\
                     '^SSMI.csv', '^BVSP.csv']
dataset_names = ['S&P 500', 'N225', 'SSE', 'HSI', 'BSESN', 'SMI', 'BVSP']

# specify drawdown thresholds for crashes (determined in exploration.ipynb):
crash_thresholds = [-0.0936, -0.1101, -0.1269, -0.1470, -0.1703, -0.1106, -0.2344] # <-- Jacobsson
# crash_thresholds = [-0.1053, -0.1495, -0.1706, -0.2334, -0.1563, -0.1492, -0.2264] # <-- Sornette
months = [1, 3, 6]              # <-- predict if crash n months ahead (use: 1, 3 or 6)
data = DataLoader(datasets_original, dataset_names)
datasets_revised, crashes = data.get_data_revised(crash_thresholds)
dfs_x, dfs_y = data.get_dfs_xy(months=months)

['^GSPC.csv', '^N225.csv', 'SSE.csv', '^HSI.csv', '^BSESN.csv', '^SSMI.csv', '^BVSP.csv']
csv_ read
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
1950-01-03    16.660000    16.660000    16.660000    16.660000    16.660000   
1950-01-04    16.850000    16.850000    16.850000    16.850000    16.850000   
1950-01-05    16.930000    16.930000    16.930000    16.930000    16.930000   
1950-01-06    16.980000    16.980000    16.980000    16.980000    16.980000   
1950-01-09    17.080000    17.080000    17.080000    17.080000    17.080000   
...                 ...          ...          ...          ...          ...   
2018-09-06  2888.639893  2892.050049  2867.290039  2878.050049  2878.050049   
2018-09-07  2868.260010  2883.810059  2864.120117  2871.679932  2871.679932   
2018-09-10  2881.389893  2886.929932  2875.939941  2877.129883  2877.129883   
2018-09-11  2871.570068  2892.5

csv_ read
                    Open          High           Low         Close  \
Date                                                                 
1986-12-31   2568.300049   2568.300049   2568.300049   2568.300049   
1987-01-01           NaN           NaN           NaN           NaN   
1987-01-02   2540.100098   2540.100098   2540.100098   2540.100098   
1987-01-05   2552.399902   2552.399902   2552.399902   2552.399902   
1987-01-06   2583.899902   2583.899902   2583.899902   2583.899902   
...                  ...           ...           ...           ...   
2018-09-21  27712.349609  27965.960938  27536.269531  27953.580078   
2018-09-24  27783.849609  27783.849609  27425.009766  27499.390625   
2018-09-26  27606.810547  28031.810547  27589.369141  27816.869141   
2018-09-27  27912.509766  27928.289063  27638.539063  27715.669922   
2018-09-28  27879.060547  27927.949219  27672.830078  27788.519531   

               Adj Close        Volume  
Date                                  

In [ ]:
# -------------------- Find best parameters with grid search -------------------- #
model_name = 'Logistic Regression'
test_data = 'S&P 500'
month_prediction = 3
beta = 2

index_test = [i for i, name in enumerate(dataset_names) if name == test_data][0]
index_month = [i for i, m in enumerate(months) if m == month_prediction][0]
training_set_names = list(dataset_names)
training_set_names.pop(index_test)
dfs_x_training = list(dfs_x)
dfs_x_training.pop(index_test)
dfs_y_training = list(dfs_y)
dfs_y_training.pop(index_test)
x_train, y_train, _, _ = data.get_train_test(dfs_x_training, dfs_y_training, \
            training_set_names, test_data=None)
y_train = y_train[:, index_month].astype(int)

# Find parameters with grid search:
fbeta_scorer = make_scorer(fbeta_score, beta=beta)
# param_grid = [{'C': [0.1, 1, 10, 100, 1000], 'class_weight': [{0:.05, 1:.95}, {0:.04, 1:.96}, \
#               {0:.03, 1:.97}, {0:.025, 1:.975}, {0:.02, 1:.98}]}]   # <-- 1 month
param_grid = [{'C': [0.1, 1, 10, 100, 1000], 'class_weight': [{0:.07, 1:.93}, {0:.06, 1:.94}, \
                {0:.05, 1:.95}, {0:.04, 1:.96}]}]   # <-- 3 months
# param_grid = [{'C': [0.1, 1, 10, 100, 1000], 'class_weight': [{0:.11, 1:.89}, {0:.1, 1:.9},\
#                 {0:.09, 1:.91}, {0:.08, 1:.92}]}]   # <-- 6 months

clf = GridSearchCV(linear_model.LogisticRegression(penalty='l2'), param_grid, scoring=fbeta_scorer, return_train_score=True) 
model = clf.fit(x_train, y_train)
labels = model.cv_results_['params']


tr_score = model.cv_results_['mean_train_score']
t_score = model.cv_results_['mean_test_score']
rcParams['figure.figsize'] = 8, 4
plt.bar(x=np.arange(len(tr_score)) - 0.2,width=0.4, height=tr_score, color='lightblue', label='train score')
plt.bar(x=np.arange(len(t_score)) + 0.2,width=0.4, height=t_score, color='blue', label='test score')
plt.title('Find best parameters with F beta score (beta=2)')
plt.xticks(np.arange(len(labels)), labels, rotation=90)
plt.ylabel('F-Beta score (beta=2)')
plt.legend()
plt.grid()
plt.show()



In [ ]:
# tr_score = model.cv_results_['mean_train_score']
clf.cv_results_.keys()

In [ ]:
# -------------------- Train Logistic Regression -------------------- #
class_weight = {0:.06, 1:.94}
C = 1
index_test = [i for i, name in enumerate(dataset_names) if name == test_data][0]
index_month = [i for i, m in enumerate(months) if m == month_prediction][0]
training_set_names = list(dataset_names)
training_set_names.pop(index_test)
dfs_x_training = list(dfs_x)
dfs_x_training.pop(index_test)
dfs_y_training = list(dfs_y)
dfs_y_training.pop(index_test)
y_train_all, y_val_all = [], []
y_pred_train_all, y_pred_val_all = [], []
for val_data in training_set_names:
    x_train, y_train, x_val, y_val = data.get_train_test(dfs_x_training, dfs_y_training, \
            training_set_names, test_data=val_data)
    y_train, y_val = y_train[:, index_month].astype(int), y_val[:, index_month].astype(int)
    y_train_all.append(y_train)
    y_val_all.append(y_val)
    print('Train ' + str(model_name) + ' - validation data: ' + str(val_data))
    clf = linear_model.LogisticRegression(C=C, class_weight=class_weight)
    model = clf.fit(x_train, y_train)
    y_pred_train = model.predict(x_train)
    y_pred_train_all.append(y_pred_train)
    y_pred_val = model.predict(x_val)
    y_pred_val_all.append(y_pred_val)

In [ ]:
# -------------------- Evaluate results -------------------- #
eval_ = EvaluateResults(y_train_all, y_val_all, y_pred_train_all, y_pred_val_all, model_name, test_data)
beta = 2
threshold = None
print(model_name)
print('\n')
print('Predict crash in:               ' + str(month_prediction) + ' months')
print('Number of features:             ' + str(dfs_x[0].shape[1]))
print('Number of rows in training set: ' + str(len(y_pred_train_all[0]) + len(y_pred_val_all[0])))
print('\n')
eval_.training_results(threshold, training_set_names, beta=beta)

In [ ]:
# -------------------- Test model -------------------- #
x_train, y_train, x_test, y_test = data.get_train_test(dfs_x, dfs_y, dataset_names, test_data=test_data)
y_train, y_test = y_train[:, index_month].astype(int), y_test[:, index_month].astype(int)
lm = linear_model.LogisticRegression(C=C, class_weight=class_weight)
model = lm.fit(x_train, y_train)
y_pred_test_bin = model.predict(x_test).astype(int)
threshold = None
_ = eval_.test_results(y_test, y_pred_test_bin, threshold, beta=beta)

In [ ]:

# -------------------- Plot test results -------------------- #
df = datasets_revised[index_test].reindex(dfs_x[index_test].index)
df['y'] = y_test
df['y_pred'] = y_pred_test_bin
c = crashes[index_test]
t_start = ['1956-01-01', '1971-01-01', '1976-01-01', '1983-01-01', '1995-01-01', '2004-01-01', '2010-01-01']
t_end = ['1963-01-01', '1981-01-01', '1983-01-01', '1988-01-01', '2003-01-01', '2010-01-01', '2016-01-01']
rcParams['figure.figsize'] = 10, 6
eval_.plot_test_results(df, c, t_start, t_end)

In [ ]:
# -------------------- Current prediction S&P 500 -------------------- #
# train on all available data:
os.chdir('/Users/Hp/VS CODE/Stock market crash prediction')
x_train, y_train, _, _ = data.get_train_test(dfs_x, dfs_y, dataset_names, test_data=None)
C = [1, 1, 1]
class_weights = [{0:.06, 1:.94}, {0:.06, 1:.94}, {0:.08, 1:.92}]
dataset_original = ['^GSPC_11-05.csv']
dataset_name = ['S&P 500']
crash_threshold = [-0.0936]
data_new = DataLoader(dataset_original, dataset_name)
dataset_revised, crashes = data_new.get_data_revised(crash_threshold)
dfs_x_new, dfs_y_new = data_new.get_dfs_xy_predict(months=months)
x_new, _, _, _ = data_new.get_train_test(dfs_x_new, dfs_y_new, dataset_name, test_data=None)

os.chdir('/Users/Hp/VS CODE/Stock market crash prediction')
for index_month in range(len(months)):
    y_train_ = y_train[:, index_month].astype(int)
    lm = linear_model.LogisticRegression(C=C[index_month], class_weight=class_weights[index_month])
    model = lm.fit(x_train, y_train_)
    filename = 'logreg_model_{}months.sav'.format(months[index_month])
    pickle.dump(model, open(filename, 'wb'))
    y_pred_new_bin = model.predict(x_new).astype(int)
    current_pred = np.dot(np.linspace(0,1,21) / sum(np.linspace(0,1,21)), y_pred_new_bin[-21:])
    print(str(model_name) + ' prediction of a crash within ' + str(months[index_month]) \
          + ' months: ' + str(np.round(current_pred, 2)))